In [1]:
# 우리가 쓸 재료를 준비해줍니다.
import torch

In [2]:
# 코드를 짜기 전에 앞서, 우리가 써야 하는 gpu로 설정을 해줄게요.
# 우리는 런타임 유형을 통해서 이미 gpu를 쓰겠다고 말해둔 상태지만,
# 여러분이 나중에 서버에서 모델을 돌리거나, 기타 상황 등에서 직접 gpu 설정을 해줘야 할 수도 있습니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu' # 이 코드는 cuda, 즉 gpu를 쓸 수 있는 환경이면 cuda를 쓰고, 아니면 cpu를 써라 라는 코드입니다.
# 만약 cpu로 돌리면 batch 단위로 데이터가 들어가지 못하게 될 거에요.

# 다음은 seed를 고정해줄텐데요,
# seed를 고정한다는 말은 여러분과 제 결과를 동일하게 보여준다~ 는 의미입니다.
# random seed를 고정한다는 말이 좀 더 정확한 말입니다.
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [27]:
# 그럼 이제 간단한 데이터를 넣어볼게요.
# xor 문제인데요, 지금 들어가는 값들은 좌표 평면의 값이라고 생각하시면 됩니다.

X = torch.FloatTensor([[0,0], [0,1], [1,0], [1,10]]).to(device) #0,0, 0,1, 1,0, 1,1을 tensor 형태로 입력해줄게요.
Y = torch.FloatTensor([[0], [1], [1], [0]]).cuda() #그에 대한 label, 즉 실제값도 입력해줍니다.

In [28]:
# 우리는 총 4개의 layer를 쌓을 건데요,
# 첫 번째는 입력으로 들어온 2개의 값을 10개의 output으로 보내고,
# 두 번째는 그 10개의 값을 받아서 또 10개로 내뱉고,
# 세 번째 역시 10,10,
# 마지막은 10개의 인풋을 하나의 output으로 바꿔주는 layer입니다.
# 그리고 이 1개의 값이 sigmoid라는 활성화 함수를 통과해서 최종 Output이 결정되는 것입니다.

# layer를 쌓는 과정은 이와 같이 layer를 먼저 결정해주고
linear1 = torch.nn.Linear(2, 10, bias=True)
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

In [29]:
# sequential을 이용해서 차곡차곡 쌓아주는 것입니다.
# 뒤에 심화 과정에서 배우겠지만, 대부분 모델을 쌓을 때 저렇게 먼저 재료를 준비해주고 이 재료를 sequential을 이용해 하나의 블럭을 만들어주게 됩니다.
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)

In [30]:
# 다음으로는 손실함수와 최적화 함수를 정의해줄게요.
# 여기서는 BCE, Binary Cross Entropy라는 손실함수와
# 우리가 배웠던 SGD를 이용해줄게요.
# Binary Cross Entropy는
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=10)  # 처음에는 1로 하고 0.1, 10으로 차례대로 바꾸기

In [31]:
# 자 이제 모델을 돌려보도록 하겠습니다.
# 우리는 총 10001번의 epoch을 돌릴 겁니다. 즉 모델을 10001번 학습시킨다는 이야기죠.
for step in range(10001):
    optimizer.zero_grad() # 일단 가장 먼저 해주어야 하는 것은 optimizer에 저장된 gradient 값을 0로 만들어줘야 합니다. 우리는 위에 정의한 SGD에 의해 Loss에 따른 gradient를 계산하게 되고 이를 optimizer에 저장하게 되는데요,
    # 이 부분에 대해서는 코드를 끝까지 작성하고 설명할게요.
    hypothesis = model(X) # 다음으로는 hypothesis, 즉 y hat 예측값을 구해줍니다.

    # cost/loss function
    cost = criterion(hypothesis, Y) # 그리고 y hat과 y의 오차를 구해주고요,
    cost.backward() # backward 메소드를 통해 cost, 즉 오차를 기반으로 하여 backpopagation을 진행합니다.
    optimizer.step() # Backprop 과정이 수식적인 부분이 많고, 이를 수업시간에 다 설명하기 어려워 잠깐 넘어갔는데요, 
    # 우리는 cost 를 이용해 직접적으로 weight에 영향을 주는 것이 아닌,
    # 이 오차를 기반으로 해서 backprop 과정에서 구한 gradient 값으로 weight를 업데이트 해주게 됩니다.
    # 그래서 우리는 그 gradient 값을 optimizer에 저장을 해주는데,
    # 이렇게 업데이트된 weight에 따른 예측값은 이전과 당연히 다르겠죠?
    # 그래서 제일 위에서 zero_grad라는 메소드를 optimizer에 달아준 것입니다.    

    if step % 100 == 0:
        print(step, cost.item()) # cost 출력

0 0.7121162414550781
100 1.0441728830337524
200 1.0388953685760498
300 0.4784403443336487
400 0.4779386520385742
500 0.47775983810424805
600 0.47766825556755066
700 0.47761261463165283
800 0.47757524251937866
900 0.4775484800338745
1000 0.47752830386161804
1100 0.4775125980377197
1200 0.4775000214576721
1300 0.47748976945877075
1400 0.4774811565876007
1500 0.4774739444255829
1600 0.4774676561355591
1700 0.47746217250823975
1800 0.4774574935436249
1900 0.4774532616138458
2000 0.47744959592819214
2100 0.4774462580680847
2200 0.47744327783584595
2300 0.47744056582450867
2400 0.47743815183639526
2500 0.477435827255249
2600 0.47743380069732666
2700 0.47743189334869385
2800 0.4774301052093506
2900 0.47742849588394165
3000 0.4774269461631775
3100 0.47742557525634766
3200 0.4774242639541626
3300 0.4774230718612671
3400 0.4774218797683716
3500 0.4774208068847656
3600 0.47741979360580444
3700 0.47741878032684326
3800 0.4774179458618164
3900 0.4774170219898224
4000 0.47741615772247314
4100 0.4774